In [4]:
!pip install transformers
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader, random_split, SubsetRandomSampler
from transformers import Trainer, AutoTokenizer, AutoModel, AutoConfig
from sklearn.model_selection import KFold

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

cpp bert의 last hidden state size: torch.Size([1, 7, 768])
gpt의 input size: [1, n]

In [15]:
class CppDataset(Dataset):
  def __init__(self):
    # initializing - read text
    code_df = pd.read_tsv("spoc-train.tsv")
    self.x = code_df.iloc[:, 2].values
    self.y = code_df.iloc[:, 1].values

    self.tokenized = [self.tokenize_code(code) for code in self.x] #code를 tokenize한 결과를 저장

  def __getitem__(self, index):
    inputs = {
        "input_ids" : torch.tensor(self.tokenized[index]["input_ids"], dtype = torch.long, device = device),
        "attention_mask" : torch.tensor(self.tokenized[index]["attention_mask"], dtype = torch.long, device = device)
    } #tokenized index를 tensor로 변환
    #attention..도 적용 > 이거 꼭 해야하는건가요? cpp > ps 대응할 때 필요한 것 같아보이긴합니다만 잘모르겠어요 흑흑
    targets = torch.tensor(self.y[index], dtype=torch.float32)#Pseudo code 표현
    return inputs, targets

# tokenizing code
def tokenize_code(code):
  tokenizer = BertTokenizer.from_pretrained("bert-base-cpp") #load cpp BERT tokenizer
  tokenized_code = []

  return tokenized_code

In [19]:
class C2PSClass(torch.nn.Module):
  def __init__(self):
    super().__init__()
    self.encoder = BertModel.from_pretrained("bert-base-cpp")
    self.decoder = GPT2Model.from_pretrained("gpt2")
    self.pooling = torch.nn.MaxPool2d(kernel_size=(1,2)) #2d MaxPooling layer

  def forward(self, inputs):
    encoder_outputs = self.encoder(inputs["input_ids"], attention_mask=inputs["attention_mask"])
    decoder_outputs = self.decoder(inputs["input_ids"], attention_mask=inputs["attention_mask"])
    pooled_outputs = self.pooling(encoder_outputs[0]) #MaxPooling 적용
    outputs = pooled_outputs.squeeze(2).transpose(1, 2).squeeze(2) #1D tensor
    return outputs